<a href="https://colab.research.google.com/github/lyh26x03/aml-redflags-rag/blob/main/experiment_rag_v2_display.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""experiment_rag_v2

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1vYowV_H0PHjKktn2fqphP-dmEnegB_z-

# 🔧 PART 0: SETUP（環境設定）

System Version: **v2**

History:
- v1: 基礎 RAG（Dense + BM25 + RRF → LLM）
- v2: metadata 分層、priority weighting、Pre-LLM Gate

### 0.1 安裝套件


In [ ]:
!pip install pandas==2.2.2 numpy==2.0.2 pypdf langchain-text-splitters sentence-transformers faiss-cpu rank_bm25 google-generativeai huggingface_hub --upgrade

In [ ]:
!pip install groq

### 0.2 Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 0.3 設定路徑與 API Key


In [ ]:
# --- 路徑 ---
INDEX_DIR = "/content/drive/MyDrive/AML/index_v2/v2"
EXPERIMENT_ROOT_DIR = "/content/drive/MyDrive/AML/experiments/runs"

In [ ]:
# --- Embedding 模型（fallback，正常情況從 metadata.json 讀取）---
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [ ]:
# --- 檢索參數 ---
DEFAULT_TOP_K = 5
DEFAULT_RRF_K = 60

In [ ]:
# --- 功能開關 ---
DEFAULT_USE_PRIORITY_WEIGHTING = True
DEFAULT_ENABLE_GATE = True   # Gate 預設關閉，實驗時手動開啟

In [ ]:
# --- API Key ---
from google.colab import userdata
GROQ_API_KEY = userdata.get("colab-rag-llama3-dev")
GEMINI_API_KEY = userdata.get("aml-redflag-auditable-rag")
# HF_TOKEN = userdata.get('rag-demo-qwen-test')

In [ ]:
import os

# --- 驗證 ---
assert os.path.exists(INDEX_DIR), f"❌ 索引路徑不存在: {INDEX_DIR}"
print(f"✅ Config 載入完成")
print(f"   索引路徑: {INDEX_DIR}")
print(f"   Priority Weighting: {DEFAULT_USE_PRIORITY_WEIGHTING}")
print(f"   Gate: {DEFAULT_ENABLE_GATE}")

###0.4 LLM 模型選擇

In [ ]:
print("\n選擇 LLM 模型：")
print("  [1] Llama-3.3-70B (Groq) 🚀 — 最強推理，適合最終產出")
print("  [2] Gemini Flash — 平衡型")
print("  [3] Gemini Flash Lite — 快速輕量")
print("  [4] Llama-3.1-8B (Groq) ⚡ — 檢核工具：測試資料檢索精準度")
print("  [5] Mixtral-8x7b (Groq) 🧠 — 中堅實力：測試邏輯中轉")

MODEL_CHOICE = input("輸入 1/2/3/4/5: ").strip() or "1"

LLM_CONFIG = {
    "1": {
        "provider": "groq",
        "llm_model_name": "llama-3.3-70b-versatile",
        "api_key": GROQ_API_KEY,
    },
    "2": {
        "provider": "gemini",
        "llm_model_name": "gemini-2.0-flash",
        "api_key": GEMINI_API_KEY,
    },
    "3": {
        "provider": "gemini",
        "llm_model_name": "gemini-2.0-flash-lite",
        "api_key": GEMINI_API_KEY,
    },
    "4": {
        "provider": "groq",
        "llm_model_name": "llama-3.1-8b-instant",
        "api_key": GROQ_API_KEY,
    },
    "5": {
        "provider": "groq",
        "llm_model_name": "mixtral-8x7b-32768",
        "api_key": GROQ_API_KEY,
    },
}

In [ ]:
SELECTED_CONFIG = LLM_CONFIG[MODEL_CHOICE]
print(f"✅ 已選擇：{SELECTED_CONFIG['llm_model_name']}")

### 0.5 共用 Imports

In [ ]:
import json
import re
import pickle
import numpy as np
import faiss
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Set
from enum import Enum
from dataclasses import dataclass, field

from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import jieba
import google.generativeai as genai
from groq import Groq

print("✅ 所有 imports 完成")

### 0.6 📊 實驗記錄器

In [ ]:
"""### 0.6 ✏️ 實驗設定（每次實驗前改這格）"""

# ============================================
# ✏️ 每次實驗前只改這格
# ============================================

EXPERIMENT_VERSION = "v2.1"   # ← 版本號
EXPERIMENT_TYPE = "fix"    # ← baseline / fix / tuning / ab_test / debug
EXPERIMENT_NOTE = "expand_KS"    # ← 自由備註（會寫進目錄名和 metadata）
ENABLE_LOGGING = True   # ← 是否記錄到 Google Drive

# EXPERIMENT_VERSION = "v2.0"   # ← 版本號
# EXPERIMENT_TYPE = "baseline"    # ← baseline / fix / tuning / ab_test / debug
# EXPERIMENT_NOTE = ""    # ← 自由備註（會寫進目錄名和 metadata）
# ENABLE_LOGGING = True   # ← 是否記錄到 Google Drive

# 行為開關
USE_PRIORITY_WEIGHTING = True   # ← 是否啟用 priority weighting
ENABLE_GATE = True    # ← 是否啟用 Pre-LLM Gate

print(f"📋 實驗設定: {EXPERIMENT_VERSION} / {EXPERIMENT_TYPE}")
print(f"   Note: {EXPERIMENT_NOTE or '(none)'}")
print(f"   Priority={USE_PRIORITY_WEIGHTING} | Gate={ENABLE_GATE} | Log={ENABLE_LOGGING}")

In [ ]:
"""### 0.7 📊 實驗記錄器"""

class ExperimentLogger:
    """RAG 實驗記錄器

    職責：把每次實驗的設定、結果、指標寫成 JSON 存到 Google Drive。
    不負責決定實驗參數（那是 Cell 0.6 的事）。
    """

    def __init__(
        self,
        base_dir: str = EXPERIMENT_ROOT_DIR,
        version: str = EXPERIMENT_VERSION,
        experiment_type: str = EXPERIMENT_TYPE,
        note: str = EXPERIMENT_NOTE,
    ):
        date_str = datetime.now().strftime("%Y%m%d")

        # 目錄名：v2.0_20250210_baseline_some_note
        exp_name = f"{version}_{date_str}_{experiment_type}"
        if note:
            exp_name += f"_{note}"

        self.exp_dir = Path(base_dir) / exp_name
        self.cases_dir = self.exp_dir / "cases"
        self.exp_dir.mkdir(parents=True, exist_ok=True)
        self.cases_dir.mkdir(exist_ok=True)

        self.metadata = {
            "experiment_id": exp_name,
            "version": version,
            "type": experiment_type,
            "note": note,
            "created_at": datetime.now().isoformat(),
            "status": "running",
        }

        print(f"📁 實驗目錄: {exp_name}")

    def log_config(self, config: Dict[str, Any]):
        """記錄配置"""
        with open(self.exp_dir / "config.json", "w", encoding="utf-8") as f:
            json.dump(config, f, indent=2, ensure_ascii=False)
        print(f"✅ 配置已保存")

    def log_case(
        self,
        case_id: str,
        scenario: str,
        expected: str,
        actual: str,
        passed: bool,
        retrieved_chunks: List[Dict] = None,
        llm_response: Dict = None,
        error: str = None,
    ):
        """記錄單個案例"""
        case_data = {
            "case_id": case_id,
            "scenario": scenario,
            "expected": expected,
            "actual": actual,
            "passed": passed,
            "timestamp": datetime.now().isoformat(),
        }
        if retrieved_chunks:
            case_data["retrieved_chunks"] = retrieved_chunks
        if llm_response:
            case_data["llm_response"] = llm_response
        if error:
            case_data["error"] = error

        status = "pass" if passed else "fail"
        with open(self.cases_dir / f"{case_id}_{expected}_{status}.json", "w", encoding="utf-8") as f:
            json.dump(case_data, f, indent=2, ensure_ascii=False)

        emoji = "✅" if passed else "❌"
        print(f"  {emoji} {case_id}: {expected} → {actual}")

    def log_batch_results(self, results: List[Dict[str, Any]]):
        """記錄批次結果"""
        with open(self.exp_dir / "results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"✅ 批次結果已保存")

    def log_metrics(self, metrics: Dict[str, Any]):
        """記錄指標"""
        with open(self.exp_dir / "metrics.json", "w", encoding="utf-8") as f:
            json.dump(metrics, f, indent=2, ensure_ascii=False)
        print(f"✅ 指標已保存")

    def finalize(self, summary: str = ""):
        """完成實驗"""
        self.metadata["status"] = "completed"
        self.metadata["completed_at"] = datetime.now().isoformat()
        if summary:
            self.metadata["summary"] = summary

        with open(self.exp_dir / "metadata.json", "w", encoding="utf-8") as f:
            json.dump(self.metadata, f, indent=2, ensure_ascii=False)

        print(f"\n🎉 實驗完成: {self.exp_dir.name}")
        print(f"📊 {summary}")

print("✅ ExperimentLogger 已定義")

In [ ]:
# class ExperimentLogger:
#     """RAG 實驗記錄器"""

#     def __init__(
#         self,
#         base_dir: str = EXPERIMENT_ROOT_DIR,
#         version: str = "v2.0",
#         experiment_type: str = "baseline",
#         description: str = "",
#         config: Dict[str, Any] = None,
#         auto_describe: bool = True,
#     ):
#         """初始化實驗記錄器"""
#         date_str = datetime.now().strftime("%Y%m%d")

#         if not description and auto_describe and config:
#             description = self._auto_generate_description(config, experiment_type)

#         exp_name = f"{version}_{date_str}_{experiment_type}"
#         if description:
#             exp_name += f"_{description}"

#         self.exp_dir = Path(base_dir) / exp_name
#         self.cases_dir = self.exp_dir / "cases"

#         self.exp_dir.mkdir(parents=True, exist_ok=True)
#         self.cases_dir.mkdir(exist_ok=True)

#         self.metadata = {
#             "experiment_id": exp_name,
#             "version": version,
#             "type": experiment_type,
#             "description": description,
#             "created_at": datetime.now().isoformat(),
#             "status": "running",
#         }

#         print(f"📁 實驗目錄: {exp_name}")

#     def _auto_generate_description(self, config: Dict[str, Any], exp_type: str) -> str:
#         """自動從配置生成描述"""
#         model_map = {
#             "llama-3.3-70b-versatile": "L70b",
#             "llama-3.1-8b-instant": "L8b",
#             "gemini-2.0-flash": "GemF",
#             "gemini-2.0-flash-lite": "GemFL",
#             "mixtral-8x7b-32768": "Mix8x7",
#         }
#         raw_name = config.get("llm_model_name", "unknown")
#         model_tag = model_map.get(raw_name, "other")

#         if exp_type == "baseline":
#             return model_tag
#         elif exp_type == "fix":
#             if config.get("use_priority_weighting"):
#                 return f"{model_tag}_priority_enabled"
#             return f"{model_tag}_bug_fix"
#         elif exp_type == "tuning":
#             changed = [model_tag]
#             if config.get("k", 5) != 5:
#                 changed.append(f"k{config['k']}")
#             return "_".join(changed[:3])
#         elif exp_type == "ab_test":
#             return f"{model_tag}_gate_comparison"
#         elif exp_type == "debug":
#             if "target_case" in config:
#                 return f"{model_tag}_case_{config['target_case']}"
#             return f"{model_tag}_investigation"
#         return model_tag

#     def log_config(self, config: Dict[str, Any]):
#         """記錄配置"""
#         config_path = self.exp_dir / "config.json"
#         with open(config_path, "w", encoding="utf-8") as f:
#             json.dump(config, f, indent=2, ensure_ascii=False)
#         print(f"✅ 配置已保存")

#     def log_case(
#         self,
#         case_id: str,
#         scenario: str,
#         expected: str,
#         actual: str,
#         passed: bool,
#         retrieved_chunks: List[Dict] = None,
#         llm_response: Dict = None,
#         error: str = None,
#     ):
#         """記錄單個案例"""
#         case_data = {
#             "case_id": case_id,
#             "scenario": scenario,
#             "expected": expected,
#             "actual": actual,
#             "passed": passed,
#             "timestamp": datetime.now().isoformat(),
#         }

#         if retrieved_chunks:
#             case_data["retrieved_chunks"] = retrieved_chunks
#         if llm_response:
#             case_data["llm_response"] = llm_response
#         if error:
#             case_data["error"] = error

#         status = "pass" if passed else "fail"
#         case_path = self.cases_dir / f"{case_id}_{expected}_{status}.json"

#         with open(case_path, "w", encoding="utf-8") as f:
#             json.dump(case_data, f, indent=2, ensure_ascii=False)

#         emoji = "✅" if passed else "❌"
#         print(f"  {emoji} {case_id}: {expected} → {actual}")

#     def log_batch_results(self, results: List[Dict[str, Any]]):
#         """記錄批次結果"""
#         results_path = self.exp_dir / "results.json"
#         with open(results_path, "w", encoding="utf-8") as f:
#             json.dump(results, f, indent=2, ensure_ascii=False)
#         print(f"✅ 批次結果已保存")

#     def log_metrics(self, metrics: Dict[str, Any]):
#         """記錄指標"""
#         metrics_path = self.exp_dir / "metrics.json"
#         with open(metrics_path, "w", encoding="utf-8") as f:
#             json.dump(metrics, f, indent=2, ensure_ascii=False)
#         print(f"✅ 指標已保存")

#     def finalize(self, summary: str = ""):
#         """完成實驗"""
#         self.metadata["status"] = "completed"
#         self.metadata["completed_at"] = datetime.now().isoformat()
#         if summary:
#             self.metadata["summary"] = summary

#         meta_path = self.exp_dir / "metadata.json"
#         with open(meta_path, "w", encoding="utf-8") as f:
#             json.dump(self.metadata, f, indent=2, ensure_ascii=False)

#         print(f"\n🎉 實驗完成: {self.exp_dir.name}")
#         print(f"📊 {summary}")

# print("✅ ExperimentLogger 已定義")

# 📋 PART 1: Pre-LLM Gate（知識範圍守門）

在 LLM 分析前進行 deterministic 判斷：
- 主題是否在知識範圍內？
- 是否有明確知識缺口？
- 證據是否足夠？

> Gate 是可開關的（`enable_gate` 參數），預設關閉。


In [ ]:
class KnowledgeManifest:
    """知識清單：定義系統知識範圍"""

    def __init__(self):
        self.covered_topics: Dict[str, str] = {
            "virtual_assets": "虛擬資產/加密貨幣相關紅旗",
            "cash_structuring": "現金拆分/門檻規避",
            "rapid_movement": "快速流轉/過路帳戶",
            "third_party": "第三人代辦/人頭帳戶",
            "cross_border": "跨境高風險地區",
            "identity_mismatch": "與身分/商業模式不符",
            "shell_company": "空殼公司/受益人不透明",
        }

        self.not_covered_topics: Dict[str, str] = {
            "TBML": "貿易型洗錢（Trade-Based Money Laundering）",
            "sanctions": "制裁名單篩選",
            "tax_evasion": "稅務逃漏",
        }

        self.required_evidence: Dict[str, List[str]] = {
            "TBML": ["invoice", "customs", "shipping", "goods_flow"],
        }

    def is_topic_covered(self, topic: str) -> bool:
        return topic.lower() in [t.lower() for t in self.covered_topics.keys()]

    def is_topic_explicitly_not_covered(self, topic: str) -> bool:
        return topic.upper() in [t.upper() for t in self.not_covered_topics.keys()]

    def get_required_evidence(self, topic: str) -> List[str]:
        return self.required_evidence.get(topic.upper(), [])

In [ ]:
class TopicDetector:
    """主題偵測器：rule-based 關鍵字匹配"""

    def __init__(self):
        self.topic_keywords: Dict[str, List[str]] = {
            "TBML": [
                "貿易型洗錢", "TBML", "報關", "報關單", "發票",
                "貨物", "貨物流向", "進出口", "國際貿易",
                "信用狀", "L/C", "提單", "invoice",
                "trade-based", "trade based", "customs", "shipping",
            ],
            "virtual_assets": [
                "虛擬資產", "加密貨幣", "比特幣", "以太幣",
                "混幣", "錢包", "交易所", "OTC", "私下交易",
                "非託管", "隱私幣",
            ],
            "cash_structuring": [
                "拆分", "門檻", "小額", "多筆", "申報",
            ],
            "rapid_movement": [
                "快速流轉", "入帳即轉", "過路帳戶", "多對手方",
                "很快轉出", "立即轉出", "轉出", "進出頻繁",   # 新增口語化描述
            ],
            "third_party": [
                "第三人", "代辦", "人頭", "代為操作",
            ],
            "cross_border": [
                "跨境", "境外", "高風險地區", "匯往", "匯款",
            ],
            "identity_mismatch": [
                "與身分不符", "與職業不符", "與業務不符",
                "業務無關", "不符", "無關", "無法說明",     # new
            ],
            "shell_company": [
                "空殼公司", "受益人不明", "股權結構", "不透明",
            ],
        }

    def detect_topics(self, text: str) -> Set[str]:
        detected = set()
        text_lower = text.lower()

        for topic, keywords in self.topic_keywords.items():
            for keyword in keywords:
                if keyword.lower() in text_lower:
                    detected.add(topic)
                    break

        return detected

    def detect_evidence_fields(self, text: str) -> Set[str]:
        evidence_keywords = {
            "invoice": ["發票", "invoice", "單據"],
            "customs": ["報關單", "報關", "customs", "海關"],
            "shipping": ["提單", "運送", "shipping", "物流"],
            "goods_flow": ["貨物流向", "貨物", "商品", "goods"],
        }

        detected = set()
        text_lower = text.lower()

        for field_name, keywords in evidence_keywords.items():
            for keyword in keywords:
                if keyword.lower() in text_lower:
                    detected.add(field_name)
                    break

        return detected

    def detect_explicit_knowledge_gap(self, text: str) -> bool:
        """偵測 scenario 是否明確表示知識缺口"""
        gap_patterns = [
            r"教材沒有",
            r"沒有.*章節",
            r"沒有.*資料",
            r"手上沒有",
            r"缺乏.*文件",
        ]

        for pattern in gap_patterns:
            if re.search(pattern, text):
                return True

        return False

In [ ]:
class GateDecision(Enum):
    ALLOW = "ALLOW"
    REFUSE = "REFUSE"

In [ ]:
@dataclass
class GateResult:
    decision: GateDecision
    reason_code: str = ""
    reason_message: str = ""
    metadata: Dict[str, Any] = field(default_factory=dict)

    def to_refuse_response(self) -> Dict[str, Any]:
        return {
            "scenario_summary": self.reason_message,
            "assessment": "refuse",
            "identified_flags": [],
            "follow_up_questions": [],
            "disclaimer": "本分析僅供教育參考，不構成法律意見。",
            "_gate_metadata": {
                "decision": self.decision.value,
                "reason_code": self.reason_code,
                "detected_topics": self.metadata.get("detected_topics", []),
            },
        }

In [ ]:
def pre_llm_gate(
    scenario: str,
    manifest: KnowledgeManifest = None,
    detector: TopicDetector = None,
    verbose: bool = False,
) -> GateResult:
    """
    Pre-LLM Gate（Balanced v3）

    設計哲學：
    - Gate 只負責「災難級拒絕」
    - 灰階情境一律放行，但留下明確風險標記
    - 證據不足 → 警告（warning），不是 veto
    """

    if manifest is None:
        manifest = KnowledgeManifest()
    if detector is None:
        detector = TopicDetector()

    detected_topics = detector.detect_topics(scenario)

    # ----------------------------
    # Rule 1: 使用者明確表示沒有資料
    # ----------------------------
    if detector.detect_explicit_knowledge_gap(scenario):
        uncovered = [
            t for t in detected_topics
            if manifest.is_topic_explicitly_not_covered(t)
        ]
        if uncovered:
            if verbose:
                print(f"🚫 Gate: 明確知識缺口 — {uncovered}")
            return GateResult(
                decision=GateDecision.REFUSE,
                reason_code="EXPLICIT_KNOWLEDGE_GAP",
                reason_message=f"情境明確表示缺乏 {', '.join(uncovered)} 相關教材，無法進行分析",
                metadata={
                    "detected_topics": list(detected_topics),
                    "uncovered_topics": uncovered,
                },
            )

    # ----------------------------
    # 主題分類
    # ----------------------------
    covered_topics = [
        t for t in detected_topics if manifest.is_topic_covered(t)
    ]
    explicitly_not_covered = [
        t for t in detected_topics if manifest.is_topic_explicitly_not_covered(t)
    ]

    # ----------------------------
    # Rule 2: 完全超出系統範圍（真的沒救）
    # ----------------------------
    if explicitly_not_covered and not covered_topics:
        if verbose:
            print(f"🚫 Gate: 主題完全超出範圍 — {explicitly_not_covered}")
        return GateResult(
            decision=GateDecision.REFUSE,
            reason_code="TOPIC_NOT_COVERED",
            reason_message=f"本系統未包含 {', '.join(explicitly_not_covered)} 相關教材",
            metadata={
                "detected_topics": list(detected_topics),
                "explicitly_not_covered": explicitly_not_covered,
            },
        )

    # ----------------------------
    # Rule 3: 證據不足 → 風險標記（不直接拒絕）
    # ----------------------------
    evidence_warnings = []
    present_evidence = detector.detect_evidence_fields(scenario)

    for topic in detected_topics:
        required = manifest.get_required_evidence(topic)
        if not required:
            continue

        missing = set(required) - present_evidence
        missing_ratio = len(missing) / len(required)

        if missing_ratio > 0:
            evidence_warnings.append({
                "topic": topic,
                "missing_evidence": list(missing),
                "missing_ratio": round(missing_ratio, 2),
                "severity": (
                    "high" if missing_ratio > 0.5
                    else "medium" if missing_ratio > 0.25
                    else "low"
                )
            })

    if verbose and evidence_warnings:
        print(f"⚠️ Gate Warning: 證據不足標記 — {evidence_warnings}")

    # ----------------------------
    # ALLOW（附帶風險地圖）
    # ----------------------------
    if verbose:
        print(f"✅ Gate: ALLOW — Covered topics: {covered_topics}")

    return GateResult(
        decision=GateDecision.ALLOW,
        metadata={
            "detected_topics": list(detected_topics),
            "covered_topics": covered_topics,
            "explicitly_not_covered": explicitly_not_covered,
            "evidence_warnings": evidence_warnings,
            "gate_mode": "balanced_v3"
        },
    )


In [ ]:
# old version
# def pre_llm_gate(
#     scenario: str,
#     manifest: KnowledgeManifest = None,
#     detector: TopicDetector = None,
#     verbose: bool = False,
# ) -> GateResult:
#     """
#     Pre-LLM Gate：在 LLM 分析前進行 deterministic 判斷

#     Rules:
#         1. 明確知識缺口 → REFUSE
#         2. 主題完全超出範圍 → REFUSE
#         3. 證據不足（特定主題）→ REFUSE
#         4. 其他 → ALLOW

#     Args:
#         scenario: 情境描述
#         manifest: 知識清單（None 時使用預設）
#         detector: 主題偵測器（None 時使用預設）
#         verbose: 是否顯示詳細過程

#     Returns:
#         GateResult: 包含 decision, reason_code, reason_message, metadata
#     """
#     if manifest is None:
#         manifest = KnowledgeManifest()
#     if detector is None:
#         detector = TopicDetector()

#     # Rule 1: 明確知識缺口
#     if detector.detect_explicit_knowledge_gap(scenario):
#         detected_topics = detector.detect_topics(scenario)
#         uncovered = [
#             t for t in detected_topics
#             if manifest.is_topic_explicitly_not_covered(t)
#         ]

#         if uncovered:
#             if verbose:
#                 print(f"🚫 Gate: 明確知識缺口 — {uncovered}")

#             return GateResult(
#                 decision=GateDecision.REFUSE,
#                 reason_code="EXPLICIT_KNOWLEDGE_GAP",
#                 reason_message=f"情境明確表示缺乏 {', '.join(uncovered)} 相關教材，無法進行分析",
#                 metadata={
#                     "detected_topics": list(detected_topics),
#                     "uncovered_topics": uncovered,
#                 },
#             )

#     # Rule 2: 主題超出範圍
#     detected_topics = detector.detect_topics(scenario)
#     explicitly_not_covered = [
#         t for t in detected_topics
#         if manifest.is_topic_explicitly_not_covered(t)
#     ]
#     covered_topics_detected = [
#         t for t in detected_topics
#         if manifest.is_topic_covered(t)
#     ]

#     if explicitly_not_covered and not covered_topics_detected:
#         if verbose:
#             print(f"🚫 Gate: 主題超出範圍 — {explicitly_not_covered}")

#         return GateResult(
#             decision=GateDecision.REFUSE,
#             reason_code="TOPIC_NOT_COVERED",
#             reason_message=f"本系統未包含 {', '.join(explicitly_not_covered)} 相關教材",
#             metadata={
#                 "detected_topics": list(detected_topics),
#                 "explicitly_not_covered": explicitly_not_covered,
#             },
#         )

#     # Rule 3: 證據不足
#     for topic in detected_topics:
#         required = manifest.get_required_evidence(topic)
#         if required:
#             present_evidence = detector.detect_evidence_fields(scenario)
#             missing = set(required) - present_evidence

#             if len(missing) > len(required) / 2:
#                 if verbose:
#                     print(f"🚫 Gate: 證據不足 — {topic}")

#                 return GateResult(
#                     decision=GateDecision.REFUSE,
#                     reason_code="INSUFFICIENT_EVIDENCE",
#                     reason_message=f"{topic} 分析需要更多證據資訊",
#                     metadata={
#                         "topic": topic,
#                         "missing_evidence": list(missing),
#                     },
#                 )

#     if verbose:
#         print(f"✅ Gate: ALLOW — {detected_topics}")

#     return GateResult(
#         decision=GateDecision.ALLOW,
#         metadata={"detected_topics": list(detected_topics)},
#     )


# print("✅ PART 1: Pre-LLM Gate 模組已載入")

# 🔍 PART 2: 檢索模組 (Retrieval Pipeline)

職責：Read-Only
- 載入已建立的索引
- 提供 Dense Search（向量檢索）
- 提供 BM25 Search（關鍵字檢索）
- 提供 Hybrid Search（混合檢索）

執行時機：
- 每次查詢時
- 不需要重建索引，只需載入
"""

### 2.0 載入索引

In [ ]:
def load_all_indexes(index_dir: str) -> dict:
    """
    載入所有索引和資料

    Args:
        index_dir: 索引存放路徑

    Returns:
        dict: faiss_index, chunks, bm25_index, tokenized_corpus,
              embedding_model, metadata

    Raises:
        FileNotFoundError: 如果索引檔案不存在
        ValueError: 如果 FAISS 向量數與 chunks 數不一致
    """
    base_path = Path(index_dir)

    required_files = [
        "faiss_index.bin", "chunks.json",
        "bm25_index.pkl", "tokenized_corpus.pkl", "metadata.json",
    ]

    missing_files = [f for f in required_files if not (base_path / f).exists()]
    if missing_files:
        raise FileNotFoundError(
            f"缺少索引檔案: {missing_files}\n"
            f"請先執行 build_data notebook 建立索引。"
        )

    print(f"📂 載入索引從: {index_dir}")

    # 1. Metadata（先載入以獲取模型名稱）
    with open(base_path / "metadata.json", "r", encoding="utf-8") as f:
        metadata = json.load(f)
    print(f"   ✅ metadata.json (version: {metadata.get('version', 'unknown')})")
    print(f"      建立時間: {metadata.get('created_at', 'unknown')}")
    print(f"      Chunk Size: {metadata.get('config', {}).get('chunk_size', 'unknown')}")

    # 2. FAISS Index
    faiss_index = faiss.read_index(str(base_path / "faiss_index.bin"))
    print(f"   ✅ faiss_index.bin ({faiss_index.ntotal} vectors, dim={faiss_index.d})")

    # 3. Chunks
    with open(base_path / "chunks.json", "r", encoding="utf-8") as f:
        chunks = json.load(f)
    print(f"   ✅ chunks.json ({len(chunks)} chunks)")

    # 4. BM25 Index
    with open(base_path / "bm25_index.pkl", "rb") as f:
        bm25_index = pickle.load(f)
    print(f"   ✅ bm25_index.pkl")

    # 5. Tokenized Corpus
    with open(base_path / "tokenized_corpus.pkl", "rb") as f:
        tokenized_corpus = pickle.load(f)
    print(f"   ✅ tokenized_corpus.pkl")

    # 6. Embedding Model（從 metadata 讀取模型名稱）
    # [BUG FIX] 原本讀 metadata.get("embedding_model_name")
    # 但 build_data 寫入的 key 是 config.embedding_model
    embedding_model_name = metadata.get("config", {}).get(
        "embedding_model", EMBEDDING_MODEL_NAME
    )
    embedding_model = SentenceTransformer(embedding_model_name)
    print(f"   ✅ embedding_model ({embedding_model_name})")

    # 驗證一致性
    assert faiss_index.ntotal == len(chunks), (
        f"❌ 資料不一致：FAISS 有 {faiss_index.ntotal} 個向量，"
        f"但 chunks 有 {len(chunks)} 個"
    )

    return {
        "faiss_index": faiss_index,
        "chunks": chunks,
        "bm25_index": bm25_index,
        "tokenized_corpus": tokenized_corpus,
        "embedding_model": embedding_model,
        "metadata": metadata,
    }

In [ ]:
# === 執行載入 ===
print("=" * 60)
print("🔍 PART 2: 檢索模組")
print("=" * 60)

loaded = load_all_indexes(INDEX_DIR)

In [ ]:
# 解包為模組級變數
faiss_index = loaded["faiss_index"]
chunks = loaded["chunks"]
bm25_index = loaded["bm25_index"]
tokenized_corpus = loaded["tokenized_corpus"]
embedding_model = loaded["embedding_model"]  # SentenceTransformer，用於向量檢索
index_metadata = loaded["metadata"]          # ← 改名避免跟其他 metadata 混淆

print(f"\n✅ 索引載入完成")
print(f"📦 索引版本: {index_metadata.get('version')} | chunks: {len(chunks)} | vectors: {faiss_index.ntotal}")
print("=" * 60)

### 2.1 Dense Search（向量搜尋）


In [ ]:
def dense_search(
    query: str,
    faiss_index: faiss.Index,
    chunks: list,
    embedding_model: SentenceTransformer,
    k: int = DEFAULT_TOP_K,
) -> list:
    """
    使用向量相似度進行搜尋

    Args:
        query: 查詢文字
        faiss_index: FAISS 向量索引
        chunks: chunk 列表
        embedding_model: SentenceTransformer 模型
        k: 返回的結果數量

    Returns:
        list[dict]: 每個 dict 包含 {"chunk": ChunkDict, "score": float}
    """
    query_embedding = embedding_model.encode(
        [query], normalize_embeddings=True
    )

    scores, indices = faiss_index.search(
        np.array(query_embedding, dtype="float32"), k
    )

    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(chunks):  # 防禦性檢查
            results.append({
                "chunk": chunks[idx],
                "score": float(score),
            })

    return results

### 2.2 BM25 Search（關鍵字搜尋）


In [ ]:
def bm25_search(
    query: str,
    bm25_index,
    tokenized_corpus: list,
    chunks: list,
    k: int = DEFAULT_TOP_K,
) -> list:
    """
    使用 BM25 進行關鍵字搜尋

    Args:
        query: 查詢文字
        bm25_index: BM25Okapi 索引
        tokenized_corpus: 分詞後的語料
        chunks: chunk 列表
        k: 返回的結果數量

    Returns:
        list[dict]: 每個 dict 包含 {"chunk": ChunkDict, "score": float}
    """
    # 判斷查詢語言並分詞
    if any("\u4e00" <= char <= "\u9fff" for char in query):
        query_tokens = list(jieba.cut(query))
    else:
        query_tokens = query.lower().split()

    scores = bm25_index.get_scores(query_tokens)
    top_k_indices = np.argsort(scores)[::-1][:k]

    results = []
    for idx in top_k_indices:
        results.append({
            "chunk": chunks[idx],
            "score": float(scores[idx]),
        })

    return results

### 2.3 Hybrid Search（混合搜尋 + RRF）


In [ ]:
def hybrid_search(
    query: str,
    faiss_index: faiss.Index,
    chunks: list,
    embedding_model: SentenceTransformer,
    bm25_index,
    tokenized_corpus: list,
    k: int = DEFAULT_TOP_K,
    rrf_k: int = DEFAULT_RRF_K,
    use_priority_weighting: bool = DEFAULT_USE_PRIORITY_WEIGHTING,
) -> list:
    """
    混合搜尋：Dense + BM25，使用 RRF (Reciprocal Rank Fusion) 融合排名

    Args:
        query: 查詢文字
        faiss_index: FAISS 向量索引
        chunks: chunk 列表
        embedding_model: SentenceTransformer 模型
        bm25_index: BM25 索引
        tokenized_corpus: 分詞後的語料
        k: 返回的結果數量
        rrf_k: RRF 參數（通常為 60）
        use_priority_weighting: 是否使用 retrieval_priority 加權

    Returns:
        list[dict]: 每個 dict 包含 chunk, score, raw_rrf_score, priority_weight

    History:
        - v1: 基礎 RRF
        - v2: 加入 priority_weighting（用 metadata 的 retrieval_priority 加權）
    """
    # 取得兩種搜尋結果（多取以確保融合效果）
    dense_results = dense_search(
        query, faiss_index, chunks, embedding_model, k=k * 2
    )
    bm25_results = bm25_search(
        query, bm25_index, tokenized_corpus, chunks, k=k * 2
    )

    # 建立 chunk_id → 排名
    dense_ranks = {}
    for rank, r in enumerate(dense_results, start=1):
        chunk_id = r["chunk"]["chunk_id"]
        dense_ranks[chunk_id] = rank

    bm25_ranks = {}
    for rank, r in enumerate(bm25_results, start=1):
        chunk_id = r["chunk"]["chunk_id"]
        bm25_ranks[chunk_id] = rank

    # RRF 融合
    all_chunk_ids = set(dense_ranks.keys()) | set(bm25_ranks.keys())
    rrf_scores = {}

    for chunk_id in all_chunk_ids:
        score = 0
        if chunk_id in dense_ranks:
            score += 1 / (rrf_k + dense_ranks[chunk_id])
        if chunk_id in bm25_ranks:
            score += 1 / (rrf_k + bm25_ranks[chunk_id])
        rrf_scores[chunk_id] = score

    # 權重調整
    chunk_lookup = {c["chunk_id"]: c for c in chunks}

    if use_priority_weighting:
        weighted_scores = {}
        for chunk_id, rrf_score in rrf_scores.items():
            chunk = chunk_lookup[chunk_id]
            priority = chunk.get("retrieval_priority", 1.0)
            weighted_scores[chunk_id] = rrf_score * priority
    else:
        weighted_scores = rrf_scores

    # 排序取 top-k
    sorted_chunk_ids = sorted(
        weighted_scores.keys(),
        key=lambda x: weighted_scores[x],
        reverse=True,
    )[:k]

    # 組裝結果
    results = []
    for chunk_id in sorted_chunk_ids:
        results.append({
            "chunk": chunk_lookup[chunk_id],
            "score": weighted_scores[chunk_id],
            "raw_rrf_score": rrf_scores[chunk_id],
            "priority_weight": chunk_lookup[chunk_id].get("retrieval_priority", 1.0),
        })

    return results

### 2.4 便捷函數 retrieve_contexts()

取得上下文（供 PART 3 使用）

In [ ]:
def retrieve_contexts(
    query: str,
    faiss_index: faiss.Index,
    chunks: list,
    embedding_model: SentenceTransformer,
    bm25_index,
    tokenized_corpus: list,
    k: int = DEFAULT_TOP_K,
    method: str = "hybrid",
    use_priority_weighting: bool = DEFAULT_USE_PRIORITY_WEIGHTING,
) -> list:
    """
    檢索相關上下文，供 LLM 分析使用。
    這是 PART 2 提供給 PART 3 的主要介面。

    Args:
        query: 查詢文字（通常是情境描述）
        faiss_index: FAISS 向量索引
        chunks: chunk 列表
        embedding_model: SentenceTransformer 模型
        bm25_index: BM25 索引
        tokenized_corpus: 分詞後的語料
        k: 返回的結果數量
        method: 檢索方法 ("dense", "bm25", "hybrid")
        use_priority_weighting: 是否使用 retrieval_priority 加權

    Returns:
        list[dict]: 格式化的上下文列表
    """
    if method == "dense":
        search_results = dense_search(
            query, faiss_index, chunks, embedding_model, k
        )
    elif method == "bm25":
        search_results = bm25_search(
            query, bm25_index, tokenized_corpus, chunks, k
        )
    else:  # hybrid（預設）
        search_results = hybrid_search(
            query, faiss_index, chunks, embedding_model,
            bm25_index, tokenized_corpus, k,
            use_priority_weighting=use_priority_weighting,  # ← [BUG FIX] 原本漏傳
        )

    # 轉換為 ContextDict 格式
    contexts = []
    for r in search_results:
        chunk = r["chunk"]
        contexts.append({
            "chunk_id": chunk.get("chunk_id", ""),
            "source": chunk.get("source", "Unknown"),
            "page": chunk.get("page", 0),
            "text": chunk.get("text", ""),
            "score": r["score"],
            "doc_category": chunk.get("doc_category", "unknown"),
            "explanation_style": chunk.get("explanation_style", "neutral"),
        })

    return contexts

### 2.5 【測試】快速驗證

In [ ]:
# 測試查詢  # 資料科學/實驗者 thinking
test_query = "什麼是洗錢？"

print(f"\n🧪 測試查詢: {test_query}")
print("-" * 40)

# Dense Search
print("\n📊 Dense Search 結果:")
dense_results = dense_search(
    test_query, faiss_index, chunks, embedding_model, k=3
)
for i, r in enumerate(dense_results, 1):
    print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
          f"(score: {r['score']:.3f})")

# BM25 Search
print("\n📊 BM25 Search 結果:")
bm25_results = bm25_search(
    test_query, bm25_index, tokenized_corpus, chunks, k=3
)
for i, r in enumerate(bm25_results, 1):
    print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
          f"(score: {r['score']:.3f})")

# Hybrid Search
print("\n📊 Hybrid Search 結果:")
hybrid_results = hybrid_search(
    test_query, faiss_index, chunks, embedding_model,
    bm25_index, tokenized_corpus, k=3
)
for i, r in enumerate(hybrid_results, 1):
    print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
          f"(score: {r['score']:.3f})")

print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
      f"(Score: {r['score']:.3f} | Raw: {r['raw_rrf_score']:.3f} | W: {r['priority_weight']})")
print("\n✅ PART 2 測試完成！")

In [ ]:
# # 開發者thinking
# if __name__ == "__main__":
#     # 測試查詢
#     test_query = "什麼是洗錢？"

#     print(f"\n🧪 測試查詢: {test_query}")
#     print("-" * 40)

#     # Dense Search
#     print("\n📊 Dense Search 結果:")
#     dense_results = dense_search(
#         test_query, faiss_index, chunks, embedding_model, k=3
#     )
#     for i, r in enumerate(dense_results, 1):
#         print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
#               f"(score: {r['score']:.3f})")

#     # BM25 Search
#     print("\n📊 BM25 Search 結果:")
#     bm25_results = bm25_search(
#         test_query, bm25_index, tokenized_corpus, chunks, k=3
#     )
#     for i, r in enumerate(bm25_results, 1):
#         print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
#               f"(score: {r['score']:.3f})")

#     # Hybrid Search
#     print("\n📊 Hybrid Search 結果:")
#     hybrid_results = hybrid_search(
#         test_query, faiss_index, chunks, embedding_model,
#         bm25_index, tokenized_corpus, k=3
#     )
#     for i, r in enumerate(hybrid_results, 1):
#         print(f"   [{i}] {r['chunk']['source']} p.{r['chunk']['page']} "
#               f"(score: {r['score']:.3f})")

#     print("\n✅ PART 2 測試完成！")

這 12 題是測試「檢索」功能的——能不能找到正確的頁面？

In [ ]:
RETRIEVAL_TEST_CASES = [
    # Easy
    {"id": "E1", "query": "什麼是洗錢？", "expected_page": 2, "difficulty": "easy"},
    {"id": "E2", "query": "洗錢的三種態樣是什麼？", "expected_page": 2, "difficulty": "easy"},
    {"id": "E3", "query": "銀行現金交易多少錢以上要申報？", "expected_page": 8, "difficulty": "easy"},

    # Medium
    {"id": "M1", "query": "把大筆現金拆成小筆存入銀行會有什麼法律問題？", "expected_page": 8, "difficulty": "medium"},
    {"id": "M2", "query": "開公司戶頭要帶什麼文件？", "expected_page": 5, "difficulty": "medium"},
    {"id": "M3", "query": "銀行什麼情況下會拒絕讓我開戶？", "expected_page": 6, "difficulty": "medium"},
    {"id": "M4", "query": "為什麼金融機構要做客戶審查？", "expected_page": 4, "difficulty": "medium"},

    # Hard
    {"id": "H1", "query": "學生把帳戶賣給別人會怎樣？", "expected_page": 9, "difficulty": "hard"},
    {"id": "H2", "query": "出國可以帶多少現金？超過怎麼辦？", "expected_page": 7, "difficulty": "hard"},
    {"id": "H3", "query": "高風險地區的客戶開戶會被怎麼對待？", "expected_page": 4, "difficulty": "hard"},

    # Edge case（文件中沒有的）
    {"id": "X1", "query": "用比特幣洗錢會怎樣？", "expected_page": None, "difficulty": "edge"},
    {"id": "X2", "query": "洗錢防制法是哪一年制定的？", "expected_page": None, "difficulty": "edge"},
]

In [ ]:
def evaluate_retrieval(search_func, test_cases, k=3):
    """評估 retrieval 準確度"""
    results = {"recall@1": 0, "recall@3": 0, "details": []}
    valid_cases = [tc for tc in test_cases if tc["expected_page"] is not None]

    for tc in valid_cases:
        search_results = search_func(tc["query"])
        retrieved_pages = [r["chunk"]["page"] for r in search_results[:k]]

        hit_at_1 = retrieved_pages[0] == tc["expected_page"] if retrieved_pages else False
        hit_at_3 = tc["expected_page"] in retrieved_pages[:3]

        if hit_at_1:
            results["recall@1"] += 1
        if hit_at_3:
            results["recall@3"] += 1

        results["details"].append({
            "id": tc["id"],
            "query": tc["query"],
            "expected": tc["expected_page"],
            "got": retrieved_pages[0] if retrieved_pages else None,
            "hit": "✅" if hit_at_1 else "❌"
        })

    n = len(valid_cases)
    print(f"Recall@1: {results['recall@1']}/{n} ({results['recall@1']/n*100:.0f}%)")
    print(f"Recall@3: {results['recall@3']}/{n} ({results['recall@3']/n*100:.0f}%)")

    return results

In [ ]:
# 執行評估
search_func = lambda q: hybrid_search(q, faiss_index, chunks, embedding_model, bm25_index, tokenized_corpus)
evaluate_retrieval(search_func, RETRIEVAL_TEST_CASES)

# 🤖 PART 3: LLM 分析模組

這個部分負責「把檢索到的段落交給 LLM 分析」


職責：
- 接收 PART 2 檢索到的上下文
- 組裝 Prompt
- 呼叫 LLM 進行分析
- 格式化輸出結果

**模型區分**：
- `embedding_model`: SentenceTransformer → PART 2 向量檢索
- `llm_config` / `SELECTED_CONFIG`: Gemini / Groq → PART 3 文本生成

### 3.0 初始化 LLM Client

In [ ]:
llm_client = None

if SELECTED_CONFIG["provider"] == "groq":
    llm_client = Groq(api_key=SELECTED_CONFIG["api_key"])
    print(f"🚀 Groq 實體已就緒：{SELECTED_CONFIG['llm_model_name']}")

elif SELECTED_CONFIG["provider"] == "gemini":
    genai.configure(api_key=SELECTED_CONFIG["api_key"])
    llm_client = genai.GenerativeModel(SELECTED_CONFIG["llm_model_name"])
    print(f"🌟 Gemini 實體已就緒：{SELECTED_CONFIG['llm_model_name']}")

else:
    raise ValueError(f"❌ 未知的 Provider: {SELECTED_CONFIG['provider']}")

assert llm_client is not None, "❌ LLM Client 初始化失敗"
print(f"✅ LLM Client 就緒")

In [ ]:
# ===== 推理函數 =====
def generate_response(prompt: str):
    """統一推理介面：支援 Groq 與 Gemini"""

    # 1. 處理 Groq 邏輯
    if SELECTED_CONFIG["provider"] == "groq":
        # Groq 使用的是類似 OpenAI 的 Chat Completion 格式
        chat_completion = llm_client.chat.completions.create(
            messages=[
                {"role": "user", "content": prompt}
            ],
            model=SELECTED_CONFIG["llm_model_name"],
            # 這裡可以加入 temperature 等參數
        )
        return chat_completion.choices[0].message.content.strip()

    # 2. 處理 Gemini 邏輯
    elif SELECTED_CONFIG["provider"] == "gemini":
        # Gemini 的 llm_client 是 genai.GenerativeModel 實例
        response = llm_client.generate_content(prompt)
        return response.text

    # 3. 處理舊的或未定義的邏輯 (安全網)
    else:
        return f"❌ 錯誤：未支援的 Provider ({SELECTED_CONFIG['provider']})"

# ===== 測試 =====
test_prompt = "請簡要說明什麼是 RAG 系統？"
print(f"✅ 使用模型：{SELECTED_CONFIG['llm_model_name']}")
response = generate_response(test_prompt)
print(f"回應：\n{response}")

### 3.1 System Prompt

In [ ]:
SYSTEM_PROMPT = """
你是一位「AML 紅旗助教」，專門協助金融從業人員辨識可疑交易情境中的洗錢防制紅旗指標。

## 你的角色
- 你是「教育輔助工具」，不是執法機關
- 你的輸出是「紅旗清單 + 證據引用」，不是定罪判決
- 每個判斷都要有文件依據

## 判定標準（重要！）
### CONFIRMED（確認紅旗）
- 情境中**明確描述**多個紅旗特徵
- 檢索到的文件中有**對應的定義或標準**
- 例：「連續 3 天，每次 49 萬」+ 文件提到「刻意低於申報門檻 50 萬」

### POSSIBLE（可能紅旗）
- 情境有可疑跡象，但**缺少關鍵細節**
- 例：「多筆小額交易」但沒說具體金額、頻率、是否接近門檻
- 需要追問才能確認

### UNLIKELY（不太可能）
- 情境不符合任何紅旗特徵

### REFUSE（拒絕判斷）
- 情境涉及的領域不在你的知識範圍
- 例：TBML（貿易型洗錢），但你只有虛擬資產的文件

## 你認識的紅旗類型
| ID | 名稱 | 關鍵特徵 |
|----|------|----------|
| RF-01 | 門檻拆分 | 多筆小額、接近申報門檻、規避申報 |
| RF-02 | 快速流轉 | 入帳即轉出、多對手方、過路帳戶 |
| RF-03 | 現金異常 | 現金與背景不符、突然大量現金 |
| RF-04 | 第三人代辦 | 他人操作、人頭帳戶 |
| RF-05 | 跨境高風險 | 高風險地區、無合理跨境目的 |
| RF-06 | 與身分不符 | 交易與職業/業務不符 |
| RF-07 | 虛擬資產匿名 | 混幣、非託管錢包、隱私幣 |
| RF-08 | 公司不透明 | 空殼公司、受益人不明 |

## 輸出格式（JSON）
{
  "scenario_summary": "一句話摘要",
  "assessment": "confirmed | possible | unlikely | refuse",
  "identified_flags": [
    {
      "flag_id": "RF-XX",
      "flag_name": "名稱",
      "confidence": "high | medium | low",
      "reasoning": "分析理由",
      "evidence": [{"source": "TW_GOV", "page": 8, "quote": "引用"}],
      "missing_info": ["缺少的資訊"]
    }
  ],
  "follow_up_questions": ["追問問題"],
  "disclaimer": "本分析僅供教育參考，不構成法律意見。"
}

## 判定標準
- confirmed: 明確紅旗特徵 + 證據充足
- possible: 可疑跡象 + 需要追問
- unlikely: 不符合紅旗特徵
- refuse: 超出範圍 / 無法判斷
"""

print(f"✅ SYSTEM_PROMPT 已定義 ({len(SYSTEM_PROMPT)} 字元)")

### 3.2 build_user_prompt()


In [ ]:
def build_user_prompt(scenario: str, contexts: list) -> str:
    """
    組裝 user prompt

    Args:
        scenario: 情境描述
        contexts: retrieve_contexts() 的輸出

    Returns:
        str: 完整的 user prompt
    """
    context_str = ""
    for i, ctx in enumerate(contexts, 1):
        style_tag = {
            "authoritative": "【權威來源】",
            "simplified": "【簡化說明】",
            "technical": "【技術細節】",
        }.get(ctx.get("explanation_style", "neutral"), "")

        context_str += f"""
### 段落 {i} {style_tag}
- 來源：{ctx['source']}，第 {ctx['page']} 頁
- 分類：{ctx.get('doc_category', 'unknown')}
- 內容：{ctx['text']}
---
"""

    return f"""
## 情境描述
{scenario}

## 檢索到的相關文件
{context_str}

請根據以上資訊，分析這個情境中可能存在的 AML 紅旗。

### 使用指引
- 優先引用【權威來源】的定義和標準
- 可用【簡化說明】輔助解釋
- 所有判斷必須有明確的文件依據

請以 JSON 格式輸出。
"""

### 3.3 call_llm() 選擇模型

In [ ]:
def call_llm(
    system_prompt: str,
    user_prompt: str,
    llm_config: dict,
) -> dict:
    """
    呼叫 LLM 進行分析（支援 Gemini, Groq）

    Args:
        system_prompt: 系統提示詞
        user_prompt: 使用者提示詞
        llm_config: LLM 配置 dict，包含 provider, llm_model_name, api_key

    Returns:
        dict: LLM 回應（已解析的 JSON）
    """
    provider = llm_config["provider"]
    llm_model_name = llm_config["llm_model_name"]
    api_key = llm_config["api_key"]

    # === Groq ===
    if provider == "groq":
        client = Groq(api_key=api_key)

        try:
            completion = client.chat.completions.create(
                model=llm_model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=0.1,
                response_format={"type": "json_object"},
            )
            response_text = completion.choices[0].message.content
            return json.loads(response_text)

        except Exception as e:
            print(f"❌ Groq API 錯誤: {e}")
            return {
                "assessment": "error",
                "scenario_summary": f"Groq 處理失敗: {str(e)}",
                "identified_flags": [],
                "follow_up_questions": [],
                "disclaimer": "系統錯誤",
            }

    # === Gemini ===
    elif provider == "gemini":
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel(
            model_name=llm_model_name,
            system_instruction=system_prompt,
            generation_config={"response_mime_type": "application/json"},
        )
        try:
            response = model.generate_content(user_prompt)
            return json.loads(response.text)
        except Exception as e:
            print(f"❌ Gemini API 錯誤: {e}")
            return {
                "assessment": "error",
                "scenario_summary": f"Gemini 處理失敗: {str(e)}",
                "identified_flags": [],
                "follow_up_questions": [],
                "disclaimer": "系統錯誤",
            }

    else:
        raise ValueError(f"不支援的 LLM provider: {provider}")

### 3.4 analyze_scenario()（主函數）


In [ ]:
def analyze_scenario(
    scenario: str,
    faiss_index,
    chunks: list,
    embedding_model,
    bm25_index,
    tokenized_corpus: list,
    llm_config: dict = None,
    k: int = DEFAULT_TOP_K,
    retrieval_method: str = "hybrid",
    use_priority_weighting: bool = DEFAULT_USE_PRIORITY_WEIGHTING,
    enable_gate: bool = DEFAULT_ENABLE_GATE,
    verbose: bool = True,
) -> dict:
    """
    主函數：分析一個 AML 情境

    串接所有步驟：
    0. (可選) Pre-LLM Gate 檢查
    1. 檢索相關段落
    2. 組裝 Prompt
    3. 呼叫 LLM 進行分析

    Args:
        scenario: 情境描述
        llm_config: LLM 配置（None 時使用全域 SELECTED_CONFIG）
        use_priority_weighting: 是否使用 retrieval_priority 加權（原 v2 功能）
        enable_gate: 是否啟用 Pre-LLM Gate（原 v3 功能）

    Returns:
        dict: LLM 分析結果（JSON 格式）

    History:
        - v1 (2025-01): 基礎版本
        - v2 (2025-02-09): 加入 use_priority_weighting
        - v3 (2025-02-09): 整合 enable_gate，統一為單一函數
    """
    if llm_config is None:
        llm_config = SELECTED_CONFIG  # ← [BUG FIX] 原本 v2 誤用整個 LLM_CONFIG dict

    # ===== Step 0: Pre-LLM Gate =====
    if enable_gate:
        if verbose:
            print("=" * 60)
            print("🚦 Step 0: Pre-LLM Gate 檢查...")

        gate_result = pre_llm_gate(scenario, verbose=verbose)

        if gate_result.decision == GateDecision.REFUSE:
            if verbose:
                print(f"   🚫 決策: REFUSE")
                print(f"   原因: {gate_result.reason_message}")
                print("=" * 60)
            return gate_result.to_refuse_response()

        if verbose:
            print(f"   ✅ 決策: ALLOW")

    # ===== Step 1: 檢索 =====
    if verbose:
        if not enable_gate:
            print("=" * 60)
        print("🔍 Step 1: 檢索相關段落...")

    contexts = retrieve_contexts(
        query=scenario,
        faiss_index=faiss_index,
        chunks=chunks,
        embedding_model=embedding_model,
        bm25_index=bm25_index,
        tokenized_corpus=tokenized_corpus,
        k=k,
        method=retrieval_method,
        use_priority_weighting=use_priority_weighting,
    )

    if verbose:
        print(f"   找到 {len(contexts)} 個相關段落")
        for i, ctx in enumerate(contexts, 1):
            print(
                f"   [{i}] {ctx.get('doc_category', 'unknown'):15} "
                f"| {ctx['source']} p.{ctx['page']} "
                f"(score: {ctx['score']:.3f})"
            )

    # ===== Step 2: 組裝 Prompt =====
    if verbose:
        print("\n📝 Step 2: 組裝 Prompt...")

    user_prompt = build_user_prompt(scenario, contexts)

    if verbose:
        print(f"   Prompt 長度: {len(user_prompt)} 字元")
        print(f"\n🤖 Step 3: 呼叫 LLM ({llm_config['llm_model_name']})...")

    # ===== Step 3: 呼叫 LLM =====
    result = call_llm(SYSTEM_PROMPT, user_prompt, llm_config)  # ← [BUG FIX] 統一用 SYSTEM_PROMPT

    # 附加檢索資訊（供後續診斷）
    result["_retrieved_chunks"] = [
        {"chunk_id": ctx["chunk_id"], "source": ctx["source"],
         "page": ctx["page"], "score": ctx["score"]}
        for ctx in contexts
    ]

    if verbose:
        print("   ✅ 分析完成")
        print("=" * 60)

    return result

### 3.5 pretty_print_result()


In [ ]:
def pretty_print_result(result: dict):
    """美化輸出分析結果"""
    print("\n" + "=" * 60)
    print("📋 AML 紅旗分析報告")
    print("=" * 60)

    print(f"\n📌 情境摘要：{result.get('scenario_summary', 'N/A')}")

    assessment = result.get("assessment", "unknown")
    emoji_map = {
        "confirmed": "🚨", "possible": "⚠️", "unlikely": "✅",
        "refuse": "🚫", "error": "❌",
    }
    print(f"\n🎯 綜合評估：{emoji_map.get(assessment, '❓')} {assessment.upper()}")

    flags = result.get("identified_flags", [])
    if flags:
        print(f"\n🚩 識別到的紅旗 ({len(flags)} 個)：")
        print("-" * 40)
        for flag in flags:
            conf_emoji = {"high": "🔴", "medium": "🟡", "low": "🟢"}.get(
                flag.get("confidence"), "⚪"
            )
            print(f"\n  {conf_emoji} [{flag.get('flag_id')}] {flag.get('flag_name')}")
            print(f"     分析：{flag.get('reasoning', 'N/A')}")

            for ev in flag.get("evidence", []):
                quote = ev.get("quote", "")[:50]
                print(f"     📚 {ev.get('source')} p.{ev.get('page')}: 「{quote}...」")

    questions = result.get("follow_up_questions", [])
    if questions:
        print(f"\n❓ 建議追問：")
        for i, q in enumerate(questions, 1):
            print(f"   {i}. {q}")

    print(f"\n📜 {result.get('disclaimer', '本分析僅供教育參考，不構成法律意見。')}")
    print("=" * 60)

In [ ]:
print("\n✅ PART 3: LLM 分析模組已載入")
print(f"⚙️  使用 analyze_scenario() 統一入口")
print(f"   支援參數: use_priority_weighting / enable_gate")

# 🧪 PART 4: 測試案例

這 16 題是測試「整體系統」的——LLM 能不能正確分析情境？

### 4.1 測試案例定義

In [ ]:
END_TO_END_TEST_CASES = {
    # ========== RF-01: 門檻拆分 (Structuring) ==========
    "1A": {
        "name": "門檻拆分 - confirmed",
        "scenario": """
某客戶在同一天內分 6 次臨櫃存入現金，每筆都略低於內部申報門檻。
存完後 30 分鐘內，將大部分金額分兩筆轉出到不同帳戶。
客戶被問到資金來源時，只回「朋友借我的」，無其他證明。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-01", "RF-02"],
        "required_sources": ["TW_GOV", "FATF_VA"]
    },
    "1B": {
        "name": "門檻拆分 - possible",
        "scenario": """
某客戶最近開始頻繁存現，但每次金額都不大。
客戶說自己做小生意，現金多很正常。
你只知道「最近常存現」，不知道頻率與金額分布。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-01"],
        "required_sources": ["TW_GOV"]
    },

    # ========== RF-02: 快速流轉 (Rapid Movement) ==========
    "2A": {
        "name": "快速流轉 - confirmed",
        "scenario": """
某帳戶凌晨收到一筆大額入帳，1 小時內分散轉出到 8 個不同對手方。
隔天又有另一筆大額入帳，重複相同模式。
帳戶持有人說「只是幫朋友周轉」，不清楚資金用途。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-02"],
        "required_sources": ["FATF_VA", "FATF_TBML"]
    },
    "2B": {
        "name": "快速流轉 - possible",
        "scenario": """
某帳戶收到一筆入帳後很快轉出，但你不確定轉出對象有幾個。
客戶說轉出是「付貨款」，但沒提供發票或合約。
你也不確定這是否為該客戶常態行為。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-02"],
        "required_sources": ["TW_GOV"]
    },

    # ========== RF-03: 現金密集且與客群不符 ==========
    "3A": {
        "name": "現金異常 - confirmed",
        "scenario": """
一名長期只用轉帳的小額上班族，突然開始每週大量存現。
存現後常在當日提領或立刻轉出，且拒絕說明現金來源。
其帳戶歷史與現金流量型態明顯不一致。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-03"],
        "required_sources": ["TW_GOV"]
    },
    "3B": {
        "name": "現金異常 - possible",
        "scenario": """
某客戶常以現金交易，但他說自己是做餐飲小店。
你沒有他的營收規模，也沒有現金存入與營業型態的對照。
你只知道「現金很多」。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-03"],
        "required_sources": ["TW_GOV"]
    },

    # ========== RF-04: 第三人代辦 / 人頭帳戶 ==========
    "4A": {
        "name": "第三人代辦 - confirmed",
        "scenario": """
某帳戶持有人是學生，但交易多由「叔叔」代為操作。
交易對象多為陌生公司帳戶，且資金常在入帳後迅速轉出。
當被問到受益人與資金用途時，持有人無法說明。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-04", "RF-02"],
        "required_sources": ["TW_GOV", "FATF_VA"]
    },
    "4B": {
        "name": "第三人代辦 - possible",
        "scenario": """
客戶說是「幫家人收款」，但你不知道家人身分。
帳戶有轉入轉出，但金額大小與頻率未提供。
客戶不願意留下任何聯絡資訊。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-04"],
        "required_sources": ["TW_GOV"]
    },

    # ========== RF-05: 跨境 / 高風險地區 ==========
    "5A": {
        "name": "跨境高風險 - confirmed",
        "scenario": """
某客戶在短時間內將資金分批匯往多個境外帳戶。
其中包含高風險或非合作地區，且客戶無法說明貿易或投資理由。
匯出後又很快有資金回流到其他帳戶。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-05"],
        "required_sources": ["FATF_VA", "TW_GOV"]
    },
    "5B": {
        "name": "跨境高風險 - possible",
        "scenario": """
客戶說要匯款到海外親友，但沒有提供親友關係證明。
你不知道匯款頻率，也不知道收款地區風險屬性。
客戶表示「急用錢」要求立即處理。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-05"],
        "required_sources": ["TW_GOV"]
    },

    # ========== RF-06: 與身分 / 商業模式不符 ==========
    "6A": {
        "name": "與身分不符 - confirmed",
        "scenario": """
某小型文創工作室帳戶，卻持續收取大量與其業務無關的款項。
收款後大多立即轉出至不同個人帳戶，且無合約或發票。
負責人無法說明交易對手與資金用途。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-06", "RF-02"],
        "required_sources": ["FATF_TBML", "TW_GOV"]
    },
    "6B": {
        "name": "與身分不符 - possible",
        "scenario": """
客戶說自己是自由接案者，所以收款來源很多。
你不知道這些來源是平台撥款、還是陌生個人匯款。
你也沒有客戶過往交易型態可對照。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-06"],
        "required_sources": ["TW_GOV"]
    },

    # ========== RF-07: 虛擬資產匿名性 ==========
    "7A": {
        "name": "虛擬資產匿名 - confirmed",
        "scenario": """
客戶將多筆資金轉入加密貨幣交易所後，立刻提到非託管錢包。
隔天又從不同地址把資金打回來，並說不清楚交易對象是誰。
客戶提到使用 OTC 私下交易且不留紀錄。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-07"],
        "required_sources": ["FATF_VA"]
    },
    "7B": {
        "name": "虛擬資產匿名 - possible",
        "scenario": """
客戶說自己有在買比特幣，但只是長期投資。
你只知道他有出入金交易所，卻不知道頻率與金額。
你也不知道他是否提到外部錢包或私下交易。
""",
        "expected_assessment": "possible",
        "expected_flags": ["RF-07"],
        "required_sources": ["FATF_VA", "TW_GOV"]
    },

    # ========== RF-08: 公司 / 受益所有人不透明 ==========
    "8A": {
        "name": "公司不透明 - confirmed",
        "scenario": """
某新成立公司要求開戶並快速進行大額收付款，但無明確營業內容。
負責人無法說明實際受益人，股權結構多層且難以釐清。
交易對手遍布多地，且用途敘述反覆變更。
""",
        "expected_assessment": "confirmed",
        "expected_flags": ["RF-08"],
        "required_sources": ["FATF_TBML", "TW_GOV"]
    },
    "8B": {
        "name": "TBML 超出範圍 - refuse",
        "scenario": """
客戶從事「國際貿易」，交易涉及多種貨物與國家。
你懷疑可能有貿易型洗錢（TBML），但你手上的教材沒有任何 TBML 章節。
你也沒有報關單、發票、貨物流向等資料。
""",
        "expected_assessment": "refuse",
        "expected_flags": [],
        "required_sources": []
    }
}

### 4.2 單一情境測試


In [ ]:
# 測試 confirmed 案例
print("⚙️ 本次實驗設定:")
print(f"   LLM: {SELECTED_CONFIG['llm_model_name']}")
print(f"   Priority Weighting: {DEFAULT_USE_PRIORITY_WEIGHTING}")
print(f"   Gate: {DEFAULT_ENABLE_GATE}")

In [ ]:
result = analyze_scenario(
    scenario=END_TO_END_TEST_CASES["2B"]["scenario"],
    faiss_index=faiss_index,
    chunks=chunks,
    embedding_model=embedding_model,
    bm25_index=bm25_index,
    tokenized_corpus=tokenized_corpus,
    llm_config=SELECTED_CONFIG,
    use_priority_weighting=True,
    enable_gate=False,       # ← 啟用 Gate
)
pretty_print_result(result)

In [ ]:
result = analyze_scenario(
    scenario=END_TO_END_TEST_CASES["6A"]["scenario"],
    faiss_index=faiss_index,
    chunks=chunks,
    embedding_model=embedding_model,
    bm25_index=bm25_index,
    tokenized_corpus=tokenized_corpus,
    llm_config=SELECTED_CONFIG,
    use_priority_weighting=True,
    enable_gate=False,       # ← 啟用 Gate
)
pretty_print_result(result)

In [ ]:
# 測試 confirmed 案例
result = analyze_scenario(
    scenario=END_TO_END_TEST_CASES["1A"]["scenario"],
    faiss_index=faiss_index,
    chunks=chunks,
    embedding_model=embedding_model,
    bm25_index=bm25_index,
    tokenized_corpus=tokenized_corpus,
    llm_config=SELECTED_CONFIG,
    use_priority_weighting=True,
    enable_gate=False,       # ← 啟用 Gate
)
pretty_print_result(result)

In [ ]:
# 測試 refuse 案例（含 Gate）
result = analyze_scenario(
    scenario=END_TO_END_TEST_CASES["8B"]["scenario"],
    faiss_index=faiss_index,
    chunks=chunks,
    embedding_model=embedding_model,
    bm25_index=bm25_index,
    tokenized_corpus=tokenized_corpus,
    llm_config=SELECTED_CONFIG,
    use_priority_weighting=True,
    enable_gate=True,       # ← 啟用 Gate
)
pretty_print_result(result)

### 4.3 批次測試（可選）


In [ ]:
def run_all_tests(
    test_cases: dict,
    faiss_index,
    chunks: list,
    embedding_model,
    bm25_index,
    tokenized_corpus: list,
    llm_config: dict,
    use_priority_weighting: bool = DEFAULT_USE_PRIORITY_WEIGHTING,
    enable_gate: bool = DEFAULT_ENABLE_GATE,
    enable_logging: bool = True,
    version: str = "v2.0",
    experiment_type: str = "baseline",
):
    """
    批次執行所有測試案例

    Args:
        test_cases: END_TO_END_TEST_CASES 格式的 dict
        enable_gate: 是否啟用 Pre-LLM Gate
        enable_logging: 是否啟用實驗記錄
        version: 實驗版本號
        experiment_type: 實驗類型（baseline / fix / tuning / ab_test / debug）

    Returns:
        list[dict]: 每個 dict 包含 id, passed, expected, actual, gate_decision

    History:
        - v1: 基礎批次測試（run_all_tests）
        - v2: 整合 Gate 支援（合併 run_all_tests_with_gate）
    """
    results = []
    passed_count = 0
    total_count = len(test_cases)

    # Logger 初始化
    logger = None
    if enable_logging:
        config = {
            **llm_config,
            "use_priority_weighting": use_priority_weighting,
            "enable_gate": enable_gate,
            "k": DEFAULT_TOP_K,
            "retrieval_method": "hybrid",
            "rrf_k": DEFAULT_RRF_K,
            "index_version": index_metadata.get("version", "unknown"),
        }

        logger = ExperimentLogger(
            base_dir=EXPERIMENT_ROOT_DIR,
            version=version,
            experiment_type=experiment_type,
            # config=config,
            # auto_describe=True,
        )
        logger.log_config(config)

    for test_id, test in test_cases.items():
        print(f"\n🧪 測試 {test_id}: {test['name']}")

        try:
            result = analyze_scenario(
                scenario=test["scenario"],
                faiss_index=faiss_index,
                chunks=chunks,
                embedding_model=embedding_model,
                bm25_index=bm25_index,
                tokenized_corpus=tokenized_corpus,
                llm_config=llm_config,
                use_priority_weighting=use_priority_weighting,
                enable_gate=enable_gate,
                verbose=False,
            )

            actual = result.get("assessment", "unknown")
            expected = test["expected_assessment"]
            passed = actual == expected

            if passed:
                passed_count += 1

            gate_meta = result.get("_gate_metadata", {})
            gate_decision = gate_meta.get("decision", "LLM")

            status = "✅" if passed else "❌"
            print(f"   預期: {expected} | 實際: {actual} | 來源: {gate_decision} | {status}")

            results.append({
                "id": test_id,
                "passed": passed,
                "expected": expected,
                "actual": actual,
                "gate_decision": gate_decision,
            })

            if logger:
                logger.log_case(
                    case_id=test_id,
                    scenario=test["scenario"],
                    expected=expected,
                    actual=actual,
                    passed=passed,
                    retrieved_chunks=result.get("_retrieved_chunks", []),
                    llm_response={
                        "assessment": actual,
                        "flags": result.get("identified_flags", []),
                    },
                )

        except Exception as e:
            print(f"   ❌ 錯誤: {e}")
            results.append({"id": test_id, "passed": False, "error": str(e)})

            if logger:
                logger.log_case(
                    case_id=test_id,
                    scenario=test["scenario"],
                    expected=test["expected_assessment"],
                    actual="error",
                    passed=False,
                    error=str(e),
                )

    # 統計
    accuracy = passed_count / total_count if total_count > 0 else 0
    gate_refused = sum(1 for r in results if r.get("gate_decision") == "REFUSE")

    print(f"\n📊 總結: {passed_count}/{total_count} 通過 ({accuracy:.1%})")
    if enable_gate:
        print(f"🚦 Gate 攔截: {gate_refused} 個案例")

    if logger:
        metrics = {
            "total_cases": total_count,
            "passed": passed_count,
            "failed": total_count - passed_count,
            "accuracy": accuracy,
            "gate_refused": gate_refused,
        }
        logger.log_metrics(metrics)
        logger.log_batch_results(results)
        logger.finalize(summary=f"準確率: {accuracy:.1%} ({passed_count}/{total_count})")

    return results

In [ ]:
# 執行批次測試
print("\n" + "=" * 60)
print("🚀 PART 4: 批次測試所有情境...")
print("=" * 60)

all_test_results = run_all_tests(
    test_cases=END_TO_END_TEST_CASES,
    faiss_index=faiss_index,
    chunks=chunks,
    embedding_model=embedding_model,
    bm25_index=bm25_index,
    tokenized_corpus=tokenized_corpus,
    llm_config=SELECTED_CONFIG,
    use_priority_weighting=True,
    enable_gate=True,            # ← 啟用 Gate
    enable_logging=ENABLE_LOGGING
)

print("\n✅ 所有批次測試完成！")

# PART 5: 診斷工具（可選執行）
這部分用於驗證「權重調整」是否真的改善了系統表現。
在確認有效後，可以不執行這部分。

以下情況發生時，必須進行診斷：

* 新增資料源：如果你引進了第三種資料（例如：銀行內部規章），你需要重跑診斷，確認新資料不會把 core 法規擠下去。

* 更換 Embedding 模型：如果你改用了不同廠牌的向量模型，空間分佈會變，權重可能需要微調。

* 發現新的「幻覺」：如果之後有題目該拒答卻沒拒答，就得回來這看是不是 knowledge_bridge 又爬到第一名了。

### 5.1 單一案例檢索診斷

In [ ]:
def diagnose_single_case(test_id: str, show_chunks: bool = False):
    """
    診斷單個測試案例的檢索結果
    對比有權重 vs 無權重的排序差異
    """
    test = END_TO_END_TEST_CASES[test_id]

    print(f"\n{'=' * 70}")
    print(f"🔍 診斷 {test_id}: {test['name']}")
    print(f"{'=' * 70}")

    print(f"\n📋 預期結果：")
    print(f"   Assessment: {test['expected_assessment']}")
    print(f"   Flags: {test['expected_flags']}")
    print(f"   Required Sources: {test.get('required_sources', [])}")

    # 無權重
    results_unweighted = hybrid_search(
        query=test["scenario"],
        faiss_index=faiss_index,
        chunks=chunks,
        embedding_model=embedding_model,
        bm25_index=bm25_index,
        tokenized_corpus=tokenized_corpus,
        k=5,
        use_priority_weighting=False,
    )

    print(f"\n  【無權重調整】")
    for i, r in enumerate(results_unweighted, 1):
        chunk = r["chunk"]
        cat = chunk.get("doc_category", "unknown")
        print(f"    [{i}] {cat:18} | Score: {r['score']:.4f} | {chunk.get('source')} p.{chunk.get('page')}")

    # 有權重
    results_weighted = hybrid_search(
        query=test["scenario"],
        faiss_index=faiss_index,
        chunks=chunks,
        embedding_model=embedding_model,
        bm25_index=bm25_index,
        tokenized_corpus=tokenized_corpus,
        k=5,
        use_priority_weighting=True,
    )

    print(f"\n  【有權重調整】")
    for i, r in enumerate(results_weighted, 1):
        chunk = r["chunk"]
        cat = chunk.get("doc_category", "unknown")
        priority = r.get("priority_weight", 1.0)
        raw = r.get("raw_rrf_score", 0)
        print(f"    [{i}] {cat:18} | Score: {r['score']:.4f} (raw: {raw:.4f} × {priority:.1f}) | {chunk.get('source')} p.{chunk.get('page')}")

    # 分析變化
    print(f"\n  📊 排序變化：")
    unweighted_cats = [r["chunk"].get("doc_category") for r in results_unweighted]
    weighted_cats = [r["chunk"].get("doc_category") for r in results_weighted]

    if unweighted_cats != weighted_cats:
        print(f"    ⚠️ 權重調整改變了排序")
        for i, (u_cat, w_cat) in enumerate(zip(unweighted_cats, weighted_cats), 1):
            if u_cat != w_cat:
                print(f"       排名{i}: {u_cat} → {w_cat}")
    else:
        print(f"    ✅ 權重調整沒有改變排序")

    if show_chunks:
        print(f"\n  📝 Chunk 內容預覽（加權後）：")
        for i, r in enumerate(results_weighted, 1):
            print(f"\n    【Chunk {i}】")
            print(f"    {r['chunk']['text'][:200]}...")

    return results_weighted

In [ ]:
# # 診斷幾個關鍵案例
# print("\n" + "=" * 70)
# print("🔬 PART 4-Extra: 診斷測試")
# print("=" * 70)

# diagnose_single_case("8A", show_chunks=False)
# diagnose_single_case("8B", show_chunks=False)

### 5.2 A/B 對比測試

In [ ]:
def ab_test_comparison(test_cases: dict):
    """對比有權重 vs 無權重的測試結果"""
    print(f"\n{'=' * 70}")
    print(f"⚖️ A/B 對比測試：權重調整的影響")
    print(f"{'=' * 70}")

    # 無權重版本
    print(f"\n【版本 A：無權重調整】")
    results_a = run_all_tests(
        test_cases=test_cases,
        faiss_index=faiss_index,
        chunks=chunks,
        embedding_model=embedding_model,
        bm25_index=bm25_index,
        tokenized_corpus=tokenized_corpus,
        llm_config=SELECTED_CONFIG,
        use_priority_weighting=False,
        enable_gate=False,
    )

    # 有權重版本
    print(f"\n{'=' * 70}")
    print(f"【版本 B：有權重調整】")
    results_b = run_all_tests(
        test_cases=test_cases,
        faiss_index=faiss_index,
        chunks=chunks,
        embedding_model=embedding_model,
        bm25_index=bm25_index,
        tokenized_corpus=tokenized_corpus,
        llm_config=SELECTED_CONFIG,
        use_priority_weighting=True,
        enable_gate=False,
    )

    # 對比分析
    print(f"\n{'=' * 70}")
    print(f"📊 對比分析")
    print(f"{'=' * 70}")

    pass_a = sum(1 for r in results_a if r.get("passed", False))
    pass_b = sum(1 for r in results_b if r.get("passed", False))
    total = len(test_cases)

    print(f"\n整體準確率：")
    print(f"  版本 A（無權重）: {pass_a}/{total} ({pass_a/total*100:.1f}%)")
    print(f"  版本 B（有權重）: {pass_b}/{total} ({pass_b/total*100:.1f}%)")
    print(f"  改善: {pass_b - pass_a:+d} 個案例")

    # 找出差異案例
    print(f"\n差異案例：")
    for test_id in test_cases.keys():
        result_a = next((r for r in results_a if r["id"] == test_id), None)
        result_b = next((r for r in results_b if r["id"] == test_id), None)

        if result_a and result_b:
            passed_a = result_a.get("passed", False)
            passed_b = result_b.get("passed", False)

            if passed_a != passed_b:
                status = "✅ 修正" if passed_b else "❌ 退化"
                print(f"  {test_id}: {status}")
                print(f"    版本 A: {result_a.get('actual', 'N/A')}")
                print(f"    版本 B: {result_b.get('actual', 'N/A')}")
                print(f"    預期: {test_cases[test_id]['expected_assessment']}")

    return results_a, results_b

### 5.3 執行診斷（取消註解即可執行）

In [ ]:
# diagnose_single_case("8A", show_chunks=False)
# diagnose_single_case("8B", show_chunks=False)
# ab_test_comparison(END_TO_END_TEST_CASES)

In [ ]:
# # 執行 A/B 測試（可選，因為會跑兩次所有測試）
# ab_test_comparison(END_TO_END_TEST_CASES)